# UD-Kanbun
##### 작성자 김병준(kuntakim88@gmail.com)  
漢文(Kanbun) : https://en.wikipedia.org/wiki/Kanbun  
github : https://github.com/KoichiYasuoka/UD-Kanbun  
형태소 태그 설명 : https://universaldependencies.org/u/pos/index.html  
형태소 분석기 설명 : https://universaldependencies.org/lzh/index.html


In [ ]:
!pip install udkanbun gdown
import udkanbun
import pandas as pd

In [ ]:
lzh=udkanbun.load()
# Danku=True 로 하면 -> 문장 분리 포함
# lzh=udkanbun.load(Danku=True) 

In [ ]:
# 형태소 분석 시행
s=lzh("不入虎穴不得虎子")
s

In [ ]:
# sentence_list = []
# # res = []
# for i, w in enumerate(s[1:]):
#   if w.id==1:
#     res = []
#     res.append(w)
#   else:
#     res.append(w)
#     try:
#       if s[i+2].id==1:
#         sentence_list.append(res)
#     except IndexError:
#       sentence_list.append(res)

### 형태소 분석결과 안내
https://universaldependencies.org/format.html

1.   ID: Word index, integer starting at 1 for each new sentence; may be a range for multiword tokens; may be a decimal number for empty nodes (decimal numbers can be lower than 1 but must be greater than 0).
2. FORM: Word form or punctuation symbol.
3. LEMMA: Lemma or stem of word form.
4. UPOS: Universal part-of-speech tag.
5. XPOS: Language-specific part-of-speech tag; underscore if not available.
6.FEATS: List of morphological features from the universal feature inventory or from a defined language-specific extension; underscore if not available.
7. HEAD: Head of the current word, which is either a value of ID or zero (0).
8. DEPREL: Universal dependency relation to the HEAD (root iff HEAD = 0) or a defined language-specific subtype of one.
9. DEPS: Enhanced dependency graph in the form of a list of head-deprel pairs.
10. MISC: Any other annotation.


In [ ]:
t = s[1]
print(t.id,t.form,t.lemma,t.upos,t.xpos,t.feats,t.head.id,t.deprel,t.deps,t.misc)

In [ ]:
# 형태소 겉모습
t.form

In [ ]:
# 표제어(기본 사전형 단어)
# 참고 : https://wikidocs.net/21707
t.lemma

In [ ]:
# Universial pos (범용 품사)
t.upos

In [ ]:
# Language-specific POS (해당 언어 품사)
t.xpos

In [ ]:
# 극부정 극성
t.feats

In [ ]:
# 의존 관계
t.deprel

In [ ]:
# 기타정뵤(어구 주해, 뒷칸 띄어쓰기 존재여부)
t.misc

In [ ]:
# 형태소/범용 품사 -> 동음이의어 구별 효과
print(t.form+'/'+t.upos) 

### 문장 자동분절(斷句, Danku)
참고 :  https://seg.shenshen.wiki/  
Danku=True -> 문장 분리 포함  
lzh=udkanbun.load(Danku=True) 

In [ ]:
lzhd=udkanbun.load(Danku=True)
s_danku = lzhd("孟子見梁惠王. 王曰, “叟! 不遠千里而來, 亦將有以利吾國乎?” 孟子對曰, “王! 何必曰利? 亦有仁義而已矣. 王曰, ‘何以利吾國?’ 大夫曰, ‘何以利吾家?’ 士庶人曰, ‘何以利吾身?’ 上下交征利而國危矣. 萬乘之國, 弑其君者, 必千乘之家, 千乘之國, 弑其君者, 必百乘之家. 萬取千焉, 千取百焉, 不爲不多矣. 苟爲後義而先利, 不奪不饜. 未有仁而遺其親者也, 未有義而後其君者也. 王亦曰仁義而已矣, 何必曰利?")
s_danku

In [ ]:
sentence_list = []
res = []
for i, w in enumerate(s_danku[1:]):
  if w.id==1:
    res = []
    res.append(w)
  else:
    res.append(w)
    try:
      if s_danku[i+2].id==1:
        sentence_list.append(res)
    except IndexError:
      sentence_list.append(res)

In [ ]:
sentence_list

### 품사를 고려해서 추출
##### 동음이의어, 하나의 단어에 여러 품사활용 등

In [ ]:
# 學/VERB vs 學/NOUN
print(lzh('學而時習之，不亦說乎？'))
print(lzh('君子不重則不威，學則不固。'))

In [ ]:
# 형태소 / 품사 형태로 추출
[t.form+'/'+t.upos for t in lzh('孟子對曰, “王! 何必曰利? 亦有仁義而已矣.')]

In [ ]:
# 원하는 품사 태그로 형태소 분석
def tokenize_tag(sentence,allow_pos=[]):
  s = lzh(sentence)
  if allow_pos !=[]:
    res = [t.form+'/'+t.upos for t in s if t.upos in allow_pos]
  else:
    res = [t.form+'/'+t.upos for t in s]
  return res

In [ ]:
tokenize_tag('孟子對曰, “王! 何必曰利? 亦有仁義而已矣',['NOUN','PROPN','VERB','ADV', 'ADJ']) #명사, 대명사, 동사, 부사, 형용사만 추출

### 논어 텍스트 형태소 분석 실습

In [ ]:
# 참고 : https://hansonminlearning.tistory.com/34
# https://docs.google.com/spreadsheets/d/13EXU26CO71UVzGiymbRed4jqT7HhrbHI/
!gdown --id 13EXU26CO71UVzGiymbRed4jqT7HhrbHI --output analects_sent.xlsx # 논어 데이터를 쓰고 싶으면 실행

In [ ]:
df = pd.read_excel('analects_sent.xlsx')

In [ ]:
df

In [ ]:
# sentence 빈칸인 행 제외
df['sentence'].dropna()

In [ ]:
df['token'] = df['sentence'].dropna().map(lambda x:tokenize_tag(x,['NOUN','PROPN','VERB','ADV', 'ADJ']))

In [ ]:
df['token']